# Disclaimer & Copyright

Copyright 2024 Forusone : shins777@gmail.com

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at

https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

# Gemini - Chat operation
* This notebook shows you how to send chat prompts to the Gemini
You can interact with Gemini using a single-turn prompt and response or chat with it in a multi-turn, continuous conversation, even for code understanding and generation.
* Refer to https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/send-chat-prompts-gemini



# Configuration
## Install python packages
* Vertex AI SDK for Python
  * https://cloud.google.com/python/docs/reference/aiplatform/latest

In [ ]:
%pip install --upgrade --quiet google-cloud-aiplatform

In [ ]:
from IPython.display import display, Markdown

## Authentication to access to the GCP & Google drive

* Use OAuth to access the GCP environment.
 * Refer to the authentication methods in GCP : https://cloud.google.com/docs/authentication?hl=ko

In [ ]:
#  For only colab to authenticate to get an access to the GCP.
import sys

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

* Mount to the google drive to access the .ipynb files in the repository.



In [ ]:
# To access contents in Google drive

if "google.colab" in sys.modules:
  from google.colab import drive
  drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Execute the example
## Set the environment on GCP Project
* Configure project information
  * Model name : LLM model name : https://cloud.google.com/vertex-ai/generative-ai/docs/learn/models
  * Project Id : prodect id in GCP
  * Region : region name in GCP

In [ ]:
MODEL_NAME="gemini-1.5-flash"
PROJECT_ID="ai-hangsik"
REGION="asia-northeast3"

### Vertex AI initialization
Configure Vertex AI and access to the foundation model.
* Vertex AI initialization : aiplatform.init(..)
  * https://cloud.google.com/python/docs/reference/aiplatform/latest#initialization

In [ ]:
import vertexai
from vertexai.generative_models import GenerativeModel, Part
import vertexai.generative_models as generative_models

# Initalizate the current vertex AI execution environment.
vertexai.init(project=PROJECT_ID, location=REGION)


system_instructions = [
 "You are an AI Specialist",
 "Answer to the question in detail"
]

# Access to the generative model.
model = GenerativeModel(MODEL_NAME,
                        system_instruction=system_instructions)

### Function to get the response

In [ ]:
from vertexai.generative_models import ChatSession

def get_chat_response(chat: ChatSession, prompt: str) -> str:
    """
    Generate a response from the model.

    chat : ChatSession
      Chat session object

    prompt : str
      The prompt to send to the model.

    Returns: str
      The generated response.

    """

    # Set model parameter : https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/send-multimodal-prompts#set_model_parameters
    generation_config = {
        "max_output_tokens": 8192,
        "temperature": 1,
        "top_p": 0.95,
    }

    # Configure satey setting : https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/configure-safety-attributes
    # Refer to the link to remove : https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/configure-safety-attributes#how_to_remove_automated_response_blocking_for_select_safety_attributes
    safety_settings = {
        generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
        generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
        generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
        generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    }

    text_response = []

    # https://cloud.google.com/python/docs/reference/aiplatform/latest/vertexai.language_models.ChatSession
    responses = chat.send_message(
        prompt,
        generation_config = generation_config,
        safety_settings = safety_settings,
        tools = None,
        stream=False
    )

    return responses.text


## Run example

In [ ]:
from time import perf_counter

def interactive_chat(chat, question:str)->str:
  """
  Interactive chat with the model.
  """

  t1_start = perf_counter()

  answer = get_chat_response(chat, question)

  display(Markdown(f"Question : {question}"))
  display(Markdown(f"Answer : {answer}"))

  t1_end  = perf_counter()
  print(f"Time : {t1_end - t1_start} seconds\n\n")


chat = model.start_chat()
interactive_chat(chat, "Hello.")
interactive_chat(chat, "What are all the colors in a rainbow?.")
interactive_chat(chat, "Why does it appear when it rains?.")



Question : Hello.

Answer : Hello! 👋 

How can I help you today? 😊  I'm an AI specialist, so feel free to ask me anything about AI, technology, or anything else that sparks your curiosity. 


Time : 2.184557699999914 seconds




Question : What are all the colors in a rainbow?.

Answer : You got it! A rainbow has seven distinct colors:

1. **Red**
2. **Orange**
3. **Yellow**
4. **Green**
5. **Blue**
6. **Indigo** 
7. **Violet**

You can remember the order using the acronym **ROYGBIV**. 🌈 

Do you want to know more about rainbows?  Perhaps how they form or some interesting facts?  I'm ready for your next question! 


Time : 1.5606930689996261 seconds




Question : Why does it appear when it rains?.

Answer : That's a great question! Rainbows appear when sunlight interacts with water droplets in the air, like after a rain shower. Here's how it works:

1. **Sunlight:**  Sunlight is made up of all the colors of the rainbow, but we see it as white light.
2. **Water Droplets:**  When sunlight hits a raindrop, it enters the droplet and bends (refracts).  Each color of light bends at a slightly different angle.
3. **Reflection:** Inside the raindrop, the light bounces off the back of the droplet.
4. **More Refraction:**  As the light leaves the raindrop, it bends again, separating the colors even further.
5. **Our Eyes:**  Since each color of light is now traveling at a slightly different angle, we see them as a distinct band of colors – a rainbow!

**Important Note:** To see a rainbow, the sun needs to be behind you and the rain in front of you. The angle between the sun, you, and the rain needs to be about 42 degrees. 

Let me know if you'd like to dive deeper into any of these steps!  😊 


Time : 2.642939802000001 seconds


